In [3]:
%run '01-04_Parabolic_Subgroup_In_Cartan_Group.ipynb'

class Irreducible_Homogeneous_Variety ( object ) :

    def __eq__ ( self , other ) :
        """Tests if two objects of the class ``Irreducible_Homogeneous_Variety`` coincide."""
        return type( other ) == self.__class__ and other.__repr__() == self.__repr__()


    def __init__( self , Parabolic_Subgroup ) :
        """
        Initialize a homogeneous variety X=G/P.

        INPUT:
        - ``Parabolic_Subgroup`` -- Parabolic_Subgroup_In_Irreducible_Cartan_Group ;

        OUTPUT: None.
        """
        assert type( Parabolic_Subgroup ) == Parabolic_Subgroup_In_Irreducible_Cartan_Group , 'The input must be a parabolic subgroup in an irreducible Cartan group.'
        self._Parabolic_Subgroup = Parabolic_Subgroup


    def __ne__ ( self , other ) :
        """Tests if two objects of the class ``Irreducible_Homogeneous_Variety`` do not coincide."""
        return not self == other


    def __str__ ( self , OutputStyle='Long' ) :
        """Returns a one-line string as short description."""
        if  OutputStyle == 'Short' : return self.Parent_Group().Cartan_String() + '/P(' + str( self.Included_Nodes() ) + ')'
        elif OutputStyle == 'Long' : return 'Smooth projective variety ' + self.__str__( OutputStyle='Short' ) + '.'
        else : return None


    def __repr__ ( self ) :
        """Returns all attributes which are necessary to initialize the object."""
        return self.Parabolic_Subgroup()


    def Available_Negative_Roots ( self ) :
        """Returns the all available negative roots."""
        for Root in self.Parent_Group().Cartan_Type().root_system().root_lattice().negative_roots() :
            if not Root in self.Parabolic_Subgroup().Negative_Roots() :
                yield Root


    def Available_Positive_Roots ( self ) :
        """Returns the all available positive roots."""
        for Root in self.Parent_Group().Cartan_Type().root_system().root_lattice().positive_roots() :
            if not Root in self.Parabolic_Subgroup().Negative_Roots() :
                yield Root


    def Available_Roots ( self ) :
        """Returns the all available roots."""
        for Root in self.Parent_Group().Cartan_Type().root_system().root_lattice().roots() :
            if not Root in self.Parabolic_Subgroup().Negative_Roots() :
                yield Root


    def Basis ( self , Label='B' ) :
        """
        Returns the basis in terms of simple roots and fundamental weights.

        INPUT:
        - ``self`` -- Parabolic_Subgroup; the parabolic subgroup P of a Cartan group G.

        OUTPUT:
        - ``Output`` -- Dictionary.

        ALGORITHM:
        - alpha_i : the simple root associated to the i-th node.
        - omega_i : the fundamental weight associated to the i-th node.

        - A = { alpha_i }
        - B_{ i : i excluded } = { alpha_i : i included } cup { omega_i : i excluded }.
          Note: B_{} = A and B_{ all i's } = C
        - C = { omega_i }
        """
        if Label in [ 'A' , 'alphas' , 'simple roots' ] :
            return dict( self.Parent_Group().Cartan_Type().root_system().weight_space().simple_roots() )
        elif Label in [ 'B' , 'mixed' , 'simple roots for included nodes and fundamental weights for excluded nodes' ] :
            MixedBasis = {}
            for Node in self.Parent_Group().Cartan_Type().index_set() :
                if Node in self.Parabolic_Subgroup().Included_Nodes() : Element = self.Parent_Group().Cartan_Type().root_system().weight_space().simple_root(Node)
                elif Node in self.Parabolic_Subgroup().Excluded_Nodes() : Element = self.Parent_Group().Cartan_Type().root_system().weight_space().fundamental_weight(Node)
                else : raise ValueError('Plausibility? The set of nodes subdivides into the disjoint union of marked ones and excluded ones.')
                MixedBasis.update( { Node : Element } )
            return MixedBasis
        elif Label in [ 'C' , 'omegas' , 'fundamental weights' ] :
            return dict( self.Parent_Group().Cartan_Type().root_system().weight_space().fundamental_weights() )
        else : raise ValueError('The input of ``Label`` is unknown.')

    # Snynoms for different functions depending on the input
    def calU ( self , *Input ) :
        """Returns the tautological subbundle, i.e. the dual of the equivariant vector bundle induced by the highest weight omega_1."""
        if len(Input) == 0 : # Synonym for the function ``Tautological_Subbundle``
            return self.Tautological_Subbundle()
        else : # Synonym for the function ``Equivariant_Vector_Bundle``
            return self.Equivariant_Vector_Bundle( Highest_Weights=list(Input) )


    def Cartan_Degree ( self ) :
        """Returns the Cartan degree of the parent Group G."""
        return self.Parent_Group().Cartan_Degree()


    def Cartan_Family ( self ) :
        """Returns the Cartan family of the parent Group G."""
        return self.Parent_Group().Cartan_Family()


    def Cartan_Matrix ( self , From='C' , To='B' ) :
        """
        Returns the base change between the bases A, B or C.

        INPUT:
        - ``self`` -- Parabolic_Subgroup; the parabolic subgroup P of a Cartan group G.

        OUTPUT:
        - ``Output`` -- Matrix.

        ALGORITHM:
        - alpha_i : the simple root associated to the i-th node.
        - omega_i : the fundamental weight associated to the i-th node.

        - A >> C : Cartan matrix
        - C >> A : Inverse of Cartan matrix (A >> C)
        - B >> C : For each excluded node i, replace the i-th column by a standard vector e_i.
        - C >> B : Inverse of B >> C
        - A >> B : Composition of A >> C >> B
        - B >> A : Inverse of A >> B
        """
        if   From in [ 'A' , 'alphas' , 'simple roots' ] : From = 'A'
        elif From in [ 'B' , 'mixed' , 'simple roots for included nodes and fundamental weights for excluded nodes' ] : From = 'B'
        elif From in [ 'C' , 'omegas' , 'fundamental weights' ] : From = 'C'
        else : raise ValueError('The input for ``From`` is inappropriate.')
        if   To   in [ 'A' , 'alphas' , 'simple roots' ] : To = 'A'
        elif To   in [ 'B' , 'mixed' , 'simple roots for included nodes and fundamental weights for excluded nodes' ] : To = 'B'
        elif To   in [ 'C' , 'omegas' , 'fundamental weights' ] : To = 'C'
        else : raise ValueError('The input for ``To`` is inappropriate.')

        if   From == To : return matrix.identity( self.Parent_Group().Cartan_Degree() )
        elif From == 'A' :
            if   To == 'B' : return self.Cartan_Matrix( From='C' , To='B') * self.Cartan_Matrix( From='A' , To='C')
            elif To == 'C' : return self.Parent_Group().Cartan_Type().cartan_matrix()
        elif From == 'B' :
            if   To == 'A' : return self.Cartan_Matrix( From='A' , To='B' ).inverse()
            elif To == 'C' :
                Matrix = copy( self.Cartan_Matrix( From='A' , To='C' ) )
                for Node in self.Parabolic_Subgroup().Excluded_Nodes() :
                    Matrix[:,Node-1] = vector([ kronecker_delta( I , Node ) for I in [ 1 .. self.Parent_Group().Cartan_Degree() ] ])
                return Matrix
        elif From == 'C' :
            if   To == 'A' : return self.Cartan_Matrix( From='A' , To='C' ).inverse()
            elif To == 'B' : return self.Cartan_Matrix( From='B' , To='C' ).inverse()
        else : raise ValueError('Plausibility? The base change could not be computed.')


    def Cartan_String ( self ) :
        """Returns the Cartan string of the parent Group G."""
        return self.Parent_Group().Cartan_String()


    def Cartan_Type ( self ) :
        """Returns the attribute ``Cartan_Type`` of the parent Group G."""
        return self.Parent_Group().Cartan_Type()


    def Dimension ( self ) :
        """Returns the dimension dim X = dimG - dimP."""
        return self.Parent_Group().Dimension() - self.Parabolic_Subgroup().Dimension()


    def Equivariant_Vector_Bundle ( self , Highest_Weights ) :
        """Returns the equivariant vector bundle induced by the given highest weights."""
        if   Highest_Weights == 0 :
            fw = self.Basis('fundamental weights')
            Highest_Weights = [ 0 * fw[1] ]
        elif not type(Highest_Weights) == list : Highest_Weights = [ Highest_Weights ]
        Components = [ Irreducible_Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety( Base_Space=self , Highest_Weight=Highest_Weight ) for Highest_Weight in Highest_Weights ]
        return Equivariant_Vector_Bundle_Over_Irreducible_Homogeneous_Variety( Components )
    # Synonym for the function ``Equivariant_Vector_Bundle``
    def VB ( self , Highest_Weights ) :
        """Returns the equivariant vector bundle induced by the given highest weights."""
        return self.Equivariant_Vector_Bundle ( Highest_Weights=Highest_Weights )


    def Excluded_Nodes ( self ) :
        """Returns the attribute ``Excluded_Nodes`` of the parabolic subgroup P."""
        return self.Parabolic_Subgroup().Excluded_Nodes()


    def Grothendieck_Group ( self ) :
        """
        Returns the Grothendieck Group of X=G/P.

        INPUT:
        - ``self`` -- Parabolic_Subgroup; the parabolic subgroup P of a Cartan group G.

        OUTPUT:
        - ``Output`` -- Integer; The rank of the Grothendieck group of X=G/P.

        ALGORITHM:
        Thanks to the post by Pieter Belmans concerning the index of partial flag varieties
        from Aug 22nd, 2018 on his blog (cf. to [Blog_PieterBelmans]_). The link is
        https://pbelmans.ncag.info/blog/2018/08/22/rank-flag-varieties/
        (Date: Apr 26th, 2021).

        It can be computed as the rank of the homology of X=G/P using the Bruhat cell decomposition,
        and is equal to #WG/#WL, where WG (resp. WL) denotes the Weyl group of G (resp. L).
        L is the Levi subgroup inside the maximal parabolic P.

        .. TODO:
        - Check if the rank of the Grothendieck group is always be given by the Euler characteristic.
        - Fix Rank_Of_GrothendieckGroup for Borel subgroups.

        REFERENCE:
        [Blog_PieterBelmans] https://pbelmans.ncag.info/blog/
        [KP2016] Kuznetsov, Alexander; Polishchuk, Alexander Exceptional collections on isotropic Grassmannians.
                 J. Eur. Math. Soc. (JEMS) 18 (2016), no. 3, 507–574.
        """
        Cartan_Type_of_Parent_Group = self.Parent_Group().Cartan_Type()
        Weyl_Group_of_Parent_Group = Cartan_Type_of_Parent_Group.root_system().root_lattice().weyl_group()
        if self.Parabolic_Subgroup().Is_Borel() :
            # Question: Because WeylGroup_of_Levi_Part.cardinality() == 1?
            return Integers()^( Weyl_Group_of_Parent_Group.cardinality() )
        else :
            Cartan_Type_of_Levi_Part = self.Parent_Group().Cartan_Type().dynkin_diagram().subtype( self.Parabolic_Subgroup().Included_Nodes() )
            Weyl_Group_of_Levi_Part = Cartan_Type_of_Levi_Part.root_system().root_lattice().weyl_group()
            return Integers()^( Weyl_Group_of_Parent_Group.cardinality() / Weyl_Group_of_Levi_Part.cardinality() )
    # Synonym for the function ``Grothendieck_Group``
    def K0 ( self ) :
        """Returns the Grothendieck Group of X=G/P."""
        return self.Grothendieck_Group()


    def Included_Nodes ( self ) :
        """Returns the attribute ``Excluded_Nodes`` of the parabolic subgroup P."""
        return self.Parabolic_Subgroup().Included_Nodes()


    def Index ( self ) :
        """
        Returns the index of X=G/P.

        INPUT:
        - ``self`` -- Parabolic_Subgroup; the parabolic subgroup P of a Cartan group G.

        OUTPUT:
        - ``Node`` -- Excluded node
        - ``Index`` -- Index associated to X = G/P(Excluded_Nodes={Node})

        ALGORITHM:
        Thanks to the post by Pieter Belmans concerning the index of partial flag varieties
        from Aug 23rd, 2018 on his blog (cf. to [Blog_PieterBelmans]_). The link is
        https://pbelmans.ncag.info/blog/2018/08/23/index-partial-flag-varieties/
        (Date: Apr 26th, 2021).

        The index, i.e. for X=G/P (P maximal), we have Pic(X) ≅ ZZ⋅O_X(1) and therefore define the index as the integer i
        such that ω∨_X ≅ O_X(1) ⊗ i.

        To compute it, we use lemma 2.19 and remark 2.20 of [KP2016]. Combined they say the following:
        Let β be the simple root corresponding to the chosen maximal parabolic subgroup P, and ξ the associated
        fundamental weight. Let ¯β be the maximal root of the same length as β such that the coefficient of β
        in the expression of ¯β is 1.
        Then the index of G/P equals i_G/P = (ρ,β+¯β)/(ξ,β).

        REFERENCE:
        [Blog_PieterBelmans] https://pbelmans.ncag.info/blog/
        [KP2016] Kuznetsov, Alexander; Polishchuk, Alexander Exceptional collections on isotropic Grassmannians.
                 J. Eur. Math. Soc. (JEMS) 18 (2016), no. 3, 507–574.
        """
        Root_Lattice = self.Parent_Group().Cartan_Type().root_system().root_lattice()
        Ambient_Space = self.Parent_Group().Cartan_Type().root_system().ambient_space()
        for Node in self.Parabolic_Subgroup().Excluded_Nodes() :
            Beta = Root_Lattice.simple_root(Node).to_ambient() # The simple root associated to ``Node``
            Xi = Ambient_Space.fundamental_weight(Node) # The fundamental weight associated to ``Node``
            Rho = Ambient_Space.rho() # Sum of fundamental weights
            Length = Beta.dot_product(Beta)
            BetaBar = [Alpha for Alpha in Root_Lattice.roots() if Alpha.coefficient(Node) == 1 and Alpha.to_ambient().dot_product(Alpha.to_ambient()) == Length][-1].to_ambient()
                # The maximal root of the same length as ``Beta`` such that the coefficient of ``Beta`` in the expression of ``BetaBar`` is 1.
            Index = Rho.dot_product(Beta + BetaBar) / Xi.dot_product(Beta)
            yield Node , Index


    def Is_Adjoint ( self ) :
        """Tests if X=G/P is adjoint (cf. grassmannian.info)"""
        assert P.Is_Maximal() , 'This function is only implemented for maximal parabolic subgroups.'
        G = self.Parent_Group()
        Cartan_Family = G.Cartan_Family()
        Cartan_Degree = G.Cartan_Degree()
        P = self.Parabolic_Subgroup()
        k = self.k()
        if Cartan_Family == 'B' and 2 <= Cartan_Degree : return k == 2
        elif Cartan_Family == 'C' and 2 <= Cartan_Degree : return k == 1
        elif Cartan_Family == 'D' and 3 <= Cartan_Degree : return k == 2 and 4 <= Cartan_Degree
        elif Cartan_Family == 'E' : return ( Cartan_Degree == 6 and k == 2 ) or ( Cartan_Degree == 7 and k == 1 ) or ( Cartan_Degree == 8 and k == 8 )
        elif Cartan_Family == 'F' : return ( Cartan_Degree == 4 and k == 1 )
        elif Cartan_Family == 'G' : return ( Cartan_Degree == 2 and k == 2 )
        else : return False


    def Is_Coadjoint ( self ) :
        """Tests if X=G/P is coadjoint (cf. grassmannian.info)"""
        assert P.Is_Maximal() , 'This function is only implemented for maximal parabolic subgroups.'
        G = self.Parent_Group()
        Cartan_Family = G.Cartan_Family()
        Cartan_Degree = G.Cartan_Degree()
        P = self.Parabolic_Subgroup()
        k = self.k()
        if Cartan_Family == 'B' and 2 <= Cartan_Degree : return k == 1
        elif Cartan_Family == 'C' and 2 <= Cartan_Degree : return k == 2
        elif Cartan_Family == 'D' and 3 <= Cartan_Degree : return k == 2 and 4 <= Cartan_Degree
        elif Cartan_Family == 'E' : return ( Cartan_Degree == 6 and k == 2 ) or ( Cartan_Degree == 7 and k == 1 ) or ( Cartan_Degree == 8 and k == 8 )
        elif Cartan_Family == 'F' : return ( Cartan_Degree == 4 and k == 4 )
        elif Cartan_Family == 'G' : return ( Cartan_Degree == 2 and k == 1 )
        else : return False


    def Is_Cominuscule ( self ) :
        """Tests if X=G/P is cominuscule (cf. grassmannian.info)"""
        assert P.Is_Maximal() , 'This function is only implemented for maximal parabolic subgroups.'
        G = self.Parent_Group()
        Cartan_Family = G.Cartan_Family()
        Cartan_Degree = G.Cartan_Degree()
        P = self.Parabolic_Subgroup()
        k = self.k()
        if Cartan_Family == 'A' : return True
        elif Cartan_Family == 'B' : return k == 1
        elif Cartan_Family == 'C' : return k == Cartan_Degree
        elif Cartan_Family == 'D' : return k in [ 1 , CartenDegree-1 , Cartan_Degree ]
        elif Cartan_Family == 'E' : return ( Cartan_Degree == 6 and k in [ 1 , 6 ] ) or ( Cartan_Degree == 7 and k == 7 )
        else : return False

    def Is_Levi_Dominant ( self , Weight ) :
        """Tests if a given weight on X=G/P is L-dominant (i.e. all coefficients over marked nodes are non-negative)."""
        assert Weight in self.Parent_Group().Cartan_Type().root_system().weight_space() , 'The weight must be an element of the weight space.'
        for Node in self.Parabolic_Subgroup().Included_Nodes() :
            if Weight.coefficient(Node) < 0 : return False
        return True


    def Is_Minuscule ( self ) :
        """Tests if X=G/P is minuscule (cf. grassmannian.info)"""
        assert P.Is_Maximal() , 'This function is only implemented for maximal parabolic subgroups.'
        G = self.Parent_Group()
        Cartan_Family = G.Cartan_Family()
        Cartan_Degree = G.Cartan_Degree()
        P = self.Parabolic_Subgroup()
        k = self.k()
        if Cartan_Family == 'A' : return True
        elif Cartan_Family == 'B' : return k == Cartan_Degree
        elif Cartan_Family == 'C' : return k == 1
        elif Cartan_Family == 'D' : return k in [ 1 , CartenDegree-1 , Cartan_Degree ]
        elif Cartan_Family == 'E' : return ( Cartan_Degree == 6 and k in [ 1 , 6 ] ) or ( Cartan_Degree == 7 and k == 7 )
        else : return False


    def Is_Regular ( self , Weight ) :
        """Tests if a given weight on X=G/P is G-regular (i.e. does not ly on a wall of a Weyl chamber)."""
        assert Weight in self.Parent_Group().Cartan_Type().root_system().weight_space() , 'The weight must be an element of the weight space.'
        return not self.Is_Singular( Weight )


    def Is_Singular ( self , Weight ) :
        """Tests if a given weight on X=G/P is G-singular (i.e. lies on a wall of a Weyl chamber)."""
        assert Weight in self.Parent_Group().Cartan_Type().root_system().weight_space() , 'The weight must be an element of the weight space.'
        # Move ``Weight`` to dominant chamber
        Weight_In_Dominant_Chamber = Weight.to_dominant_chamber()
        # Consider list of reflections of ``Weight`` and check whether ``Weight`` is invariant under some reflection
        Reflections = [ Weight_In_Dominant_Chamber.simple_reflection( Index ) for Index in self.Parent_Group().Cartan_Type().index_set() ]
        return Weight_In_Dominant_Chamber in Reflections


    def k ( self ) :
        """Returns the number ``k`` given in the realisation as Grassmannian."""
        P = self.Parabolic_Subgroup()
        assert P.Is_Maximal() , 'This function is only implemented for maximal parabolic subgroups.'
        return P.Excluded_Nodes(OutputType=list)[0]


    def N ( self ) :
        """Returns the number ``N`` given in the realisation as Grassmannian."""
        G = self.Parent_Group()
        assert G.Is_Ordinary() , 'This function is only implemented for ordinary parent groups.'
        Cartan_Family = G.Cartan_Family()
        Cartan_Degree = G.Cartan_Degree()
        if   Cartan_Family == 'A' : return Cartan_Degree + 1   # Grassmannians Gr(k,N) or Flag varieties FL( k1 , k2 , ... ; N )
        elif Cartan_Family == 'B' : return 2*Cartan_Degree + 1 # Orthogonal Grassmannians OGr(k,N) or orthogonal Flag varieties OFl( k1 , k2 , ... ; N )
        elif Cartan_Family == 'C' : return 2*Cartan_Degree     # Symplectic Grassmannians OGr(k,N) or symplectic Flag varieties SFl( k1 , k2 , ... ; N )
        elif Cartan_Family == 'D' : return 2*Cartan_Degree     # Orthogonal Grassmannians OGr(k,N) or orthogonal Flag varieties OFl( k1 , k2 , ... ; N )


    def Null_Weight ( self ) :
        """Returns the null weight of X=G/P."""
        fw = self.Basis('C')
        return 0*fw[1]


    def Parent_Group ( self ) :
        """Returns the attribute ``_Parabolic_Subgroup``."""
        return self.Parabolic_Subgroup().Parent_Group()


    def Parabolic_Subgroup ( self ) :
        """Returns the attribute ``_Parabolic_Subgroup``."""
        return self._Parabolic_Subgroup


    def Structure_Sheaf ( self , *Twists ) :
        """Returns the structure sheaf or a twist of it."""
        if len(Twists) == 0 : Twists = tuple( len(self.Excluded_Nodes())*[0] )
        else :
            assert len(Twists) == len(self.Excluded_Nodes()) , 'The input for ``Twists`` must be of the length ' + str( len(self.Excluded_Nodes()) ) +'(= number of excluded nodes).'
            for Twist_Counter , Twist in enumerate( Twists , start=1 ) :
                assert Twist in Integers() , 'The input data of the ' + str(Twist_Counter) + 'th twist is not an integer.'
        fw = self.Basis('C')
        return self.VB( Highest_Weights=[ sum([ Twists[Counter]*fw[Node] for Counter , Node in enumerate( self.Excluded_Nodes() ) ]) ] )
    # Synonym for the function ``Tautological_Subbundle``
    def calO ( self , *Twists ) :
        """Returns the structure sheaf or a twist of it."""
        return self.Structure_Sheaf( *Twists )


    def Tautological_Subbundle ( self ) :
        """Returns the tautological subbundle, i.e. the dual of the equivariant vector bundle induced by the highest weight omega_1."""
        fw = self.Basis('C')
        return self.VB( Highest_Weights=[ fw[1] ] ).Dual()


    #def SemiSimplification ( self ) :
    #    """Returns the semi-simplification Y=G/G (G thougt as parabolic subgroup)."""
    #    G = self.Parent_Group()
    #    P = G.As_Parabolic_Subgroup()
    #    return G/P

    #def Completion ( self ) :
    #    """Returns Y=G/B (B Borel subgroup in P)."""
    #    G = self.Parent_Group()
    #    B = G.BorelSubgroup()
    #    return G/B

